### 1. Load data

In [ ]:
library(tidyverse)
library(FCSplankton)
library(openxlsx)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)

unstained <- FALSE # TRUE if samples were not stained, FALSE if samples have been stained

if(unstained){
    summary <- read_csv("./unstained/summary.csv")
}else{
    stained_summary_all <- read_csv("./stained/summary.csv") # load summary data
    stained_summary <- dplyr::filter(stained_summary_all, stained_summary_all$population == "bacteria")
    unstained_summary <- read_csv("./unstained/summary.csv")}

meta <- read_csv("metadata.txt",col_types = cols(date = col_character())) 

In [ ]:
if(unstained == FALSE){
  summary <- merge(unstained_summary, stained_summary, all=TRUE)
  summary[1:3,]
}

### 2. Convert metadata

In [ ]:
meta[1:3,] # print the first few lines to know how to parse metadata

In [ ]:
# add required columns (filename, volume and comments) from metadata
file <- paste0(meta$file,".fcs") # format  sample name to filename (.fcs)
time <- meta$date
lat <- meta$lat
lon <- meta$lon
depth <- meta$depth
replicate <- meta$replicate
volume <- meta$volume
stain <- meta$stain
flag <- meta$flag
comments <- meta$comments

# add required metadata for CMAP
# time <- as.POSIXct(meta$date, format="%d/%b/%y", tz="UTC") 
# lat <- NA
# lon <- NA

# add key information from sample label
# label <- matrix(unlist(list(strsplit(meta$label, split=" "))), ncol=3, byrow=T) 
# treatment <- label[,1]
# timepoint <- label[,2]
# replicate <- label[,3]

# create new metadata
metadata <- tibble(file, time, lat, lon, depth, replicate, volume, stain, flag, comments)

### 3. Merge metadata and summary data


In [ ]:
all <- merge(summary, metadata, by="file")
all[1:3,]

### 4. Data correction
#### a. Calculate abundance

In [ ]:
all$abundance <- all$count / all$volume
all[1:3,]

#### b. Check variables before correction

In [ ]:
all %>%
    dplyr::filter(population != "beads") %>%
    ggplot(aes(abundance, -depth, col=population)) + 
    geom_point() + 
    facet_grid(population ~ lat, scale="free_x") + 
    theme_bw() +
    xlab("Abundance (cells uL-1)") + 
    ylab("Depth (m)")

In [ ]:
all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20) %>%
    group_by(lat, population) %>%
    summarize(sd = sd(scatter),
              avg=mean(scatter)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_point(size=3) +
    geom_linerange(aes(ymin=avg-sd, ymax=avg+sd)) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Scatter (normalized to beads)")

#### c. Correct bacteria counts

In [ ]:
new.all <- all

if(unstained == FALSE){

  pro <- subset(all, population == "prochloro")
  bact <- subset(all, population == "bacteria")

  for (i in 1:nrow(pro)){
    file_number <- regmatches(pro$file[i], regexpr(pattern = "[0-9].*fcs" , text = pro$file[i]))  # removes prefix from the current file so the stained and unstained files will be identical
    
    matching_file_id <- grep(file_number, bact$file) # find the file in stained samples that matches the file number
    
    id <- which(all$file == bact$file[matching_file_id] & all$population == "bacteria") # return the index of the file that matches the Pro file numbner
    
    if(length(id) !=0) new.all$abundance[id] <- all$abundance[id] - pro$abundance[i]
    
    if(length(id) !=0) new.all$count[id] <- new.all$abundance[id] * new.all$volume[i] # calculate bacteria particle count based off abundance and volume
      
     if(length(id) !=0) new.all$scatter[id] <- (((all$scatter[id]*all$abundance[id])-(pro$scatter[i]*pro$abundance[i]))/ new.all$abundance[id]) # calculate bacteria scatter: assumes adding staining does not change scatter
  }
}

new.all[1:3,]

#### d. Check corrected variables

In [ ]:
new.all %>%
    dplyr::filter(population != "beads") %>%
    ggplot(aes(abundance, -depth, col=population)) + 
    geom_point() + 
    facet_grid(population ~ lat, scale="free_x") + 
    theme_bw() +
    xlab("Abundance (cells uL-1)") + 
    ylab("Depth (m)")

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20) %>%
    group_by(lat, population) %>%
    summarize(sd = sd(scatter),
              avg=mean(scatter)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_point(size=3) +
    geom_linerange(aes(ymin=avg-sd, ymax=avg+sd)) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Scatter (normalized to beads)")

### 5. Size and carbon content conversion

In [ ]:
mie <- read.csv(system.file("scatter", paste0("calibrated-mieINFLUX.csv"),package="FCSplankton"))

mie[1:3,] ## NOTE: Leo and Penny are included in the same Mie lookup table. Choose the column index for the correct instrument that samples were run on.

In [ ]:
# find closest matches in Mie lookup table
id <- findInterval(new.all$scatter, mie$scatter)

  for(i in 1:length(id)){
      ## choose the correct column index for Influx instrument (Leo 2-7 or Penny 8-13)
      new.all$diam_mid[[i]] <- mie[id[i],2]
      new.all$diam_upr[[i]] <- mie[id[i],3]
      new.all$diam_lwr[[i]] <- mie[id[i],4]
      new.all$Qc_mid[[i]] <- mie[id[i],5]
      new.all$Qc_upr[[i]] <- mie[id[i],6]
      new.all$Qc_lwr[[i]] <- mie[id[i],7]
    }

summary[1:3,]

### 6. Plotting
#### a. Abundance profiles
##### i. Abundance surface profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20) %>%
    group_by(lat, population) %>%
    summarize(sd = sd(abundance),
              avg=mean(abundance)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_errorbar(aes(ymin=avg-sd, ymax=avg+sd), color = "black",  size = .3, width=.1) +
    geom_point(size=3) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Abundance (cells uL-1)")
 ggsave("surface_abundance.png", path = "./plots")

##### ii. Abundance depth profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=mean(abundance)) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=4) + 
    viridis::scale_colour_viridis(name="Abundance (cells uL-1)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")
 ggsave("abundance_depth_profile.png", path = "./plots")

#### b. Scatter profiles
##### i. Surface scatter profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20) %>%
    group_by(lat, population) %>%
    summarize(sd = sd(scatter),
              avg=mean(scatter)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_errorbar(aes(ymin=avg-sd, ymax=avg+sd), color = "black",  size = .3, width=.1) +
    geom_point(size=3) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Scatter (normalized to beads)")
 ggsave("surface_scatter.png", path = "./plots")

##### ii. Depth scatter profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=log(mean(scatter))) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=4) + 
    viridis::scale_colour_viridis(name="Log Scatter (normalized to beads)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")
 ggsave("scatter_depth_profile.png", path = "./plots")

#### c. Red fluorescence depth profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & population != "bacteria") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=log(mean(red))) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=4) + 
    viridis::scale_colour_viridis(name="Log Red fluorescence (normalized to beads)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")
 ggsave("red_fluorescence_depth_profile.png", path = "./plots")

#### d. Orange fluorescence depth profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & population != "bacteria") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=log(mean(orange))) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=4) + 
    viridis::scale_colour_viridis(name="Log orange fluorescence (normalized to beads)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")
ggsave("orange_fluorescence_depth_profile.png", path = "./plots")

#### e. Cell size profiles
##### i. Cell size depth profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=mean(diam_mid)) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=5) + 
    viridis::scale_colour_viridis(name="Equivalent spherical diameter\nusing mid refractive index\n(micrometer)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")
 ggsave("cell_size_depth_profile.png", path = "./plots")

##### ii. Surface cell size estimate range using different indexes of refraction

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20 & flag == 0) %>%
    group_by(lat, population) %>%
    summarize(avg=(mean(diam_mid)),
              avg_lwr=mean(diam_lwr),
              avg_upr=mean(diam_upr)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_point(size=2) +
    geom_linerange(aes(ymin=avg_lwr, ymax=avg_upr)) +
    facet_grid(population ~ ., scale="free_y") +
    theme_bw() +
    ylab("Cell size (um)")
 ggsave("surface_cell_size_RI_range.png", path = "./plots")

##### iii. Surface cell size profile using a single, specific index of refraction for each population

In [ ]:
# select specific refractive indexes for each populations
lwr <- new.all %>%
      dplyr::filter(population == "picoeuk" | population == "prochloro" | population == "synecho" | population == "bacteria") %>%
      dplyr::select(-diam_mid, -diam_upr, -Qc_mid, -Qc_upr) %>%
      dplyr::rename(cell_diameter = diam_lwr, carbon_content = Qc_lwr)

mid <- new.all %>%
      dplyr::filter(population == "unknown" | population == "beads" | population == "croco") %>%
      dplyr::select(-diam_lwr, -diam_upr, -Qc_lwr, -Qc_upr) %>%
      dplyr::rename(cell_diameter = diam_mid, carbon_content = Qc_mid)

RI.all <- merge(lwr, mid, all = TRUE)

In [ ]:
RI.all %>%
      dplyr::filter(population != "beads" & population != "unknown" & depth < 20 & flag == 0) %>%
    group_by(lat, population) %>%
    summarize(avg=(mean(cell_diameter)),
              sd=sd(cell_diameter)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_errorbar(aes(ymin=avg-sd, ymax=avg+sd), color = "black",  size = .3, width=.1) +
    geom_point(size=2) +
    facet_grid(population ~ ., scale="free_y") +
    theme_bw() +
    ylab("Cell size (um)")
 ggsave("surface_cell_size.png", path = "./plots")

#### f. Carbon content profiles
##### i. Carbon content depth profiles

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=(mean(Qc_mid))) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=5) + 
    viridis::scale_colour_viridis(name="Cellular carbon content\nusing high refractive index\n(picogram carbon per cell)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")
 ggsave("carbon_content_depth_profile.png", path = "./plots")

##### ii. Surface carbon content estimate range using different indexes of refraction

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20 & flag == 0) %>%
    group_by(lat, population) %>%
    summarize(avg=(mean(Qc_mid)),
              avg_lwr=mean(Qc_lwr),
              avg_upr=mean(Qc_upr)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_point(size=2) +
    geom_linerange(aes(ymin=avg_lwr, ymax=avg_upr)) +
    facet_grid(population ~ ., scale="free_y") +
    theme_bw() +
    ylab("Carbon content (picogram carbon per cell)")
 ggsave("surface_carbon_content_RI_range.png", path = "./plots")

##### iii. Surface (>20m) carbon content profile using a single, specific index of refraction for each population

In [ ]:
RI.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20 & flag == 0) %>%
    group_by(lat, population) %>%
    summarize(avg=(mean(carbon_content)),sd=sd(carbon_content)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_errorbar(aes(ymin=avg-sd, ymax=avg+sd), color = "black",  size = .3, width=.1) +
    geom_point(size=2) +
    facet_grid(population ~ ., scale="free_y") +
    theme_bw() +
    ylab("Carbon content (picogram carbon per cell)")
 ggsave("surface_carbon_content.png", path = "./plots")

#### g. Total biomass

In [ ]:
biomass.all <- RI.all 
biomass.all$biomass <- biomass.all$abundance * biomass.all$carbon_content

biomass.all %>%
      dplyr::filter(population != "beads" & population != "unknown" & depth == 15)  %>%
    group_by(lat, population) %>%
    summarize(avg=(mean(biomass))) %>%
    ggplot(aes(fill = population, x = lat, y = avg)) +
    geom_bar(position= "stack", stat = "identity", width=.1, color="black", size = .2) +
    ylab("Total biomass (picogram carbon per liter)")
 ggsave("total_biomass.png", path = "./plots")

### 7. Save data

In [ ]:
project <- basename(getwd())
cruise <- "MGL1704" # Cruise ID (ex. KM1906); leave blank if samples were not collected during a cruise
cruise_nickname <- "Gradients 2, Gradients 2017" # Cruise nickname commonly referred to (ex. Gradients 2, Gradients 2017); leave blank if samples were not collected during a cruise

cmap_convert(data = new.all , cruise, cruise_nickname, project, version = "v1.0")